In [1]:
!pip install --upgrade geopandas
!pip install --upgrade pyshp
!pip install --upgrade shapely
!pip install --upgrade rasterio
!pip install --upgrade rasterstats
!pip install --upgrade wget
!pip install --upgrade pygeos

Requirement already up-to-date: geopandas in /usr/local/lib/python3.6/dist-packages (0.8.1)
Requirement already up-to-date: pyshp in /usr/local/lib/python3.6/dist-packages (2.1.2)
Requirement already up-to-date: shapely in /usr/local/lib/python3.6/dist-packages (1.7.1)
Requirement already up-to-date: rasterio in /usr/local/lib/python3.6/dist-packages (1.1.6)
Requirement already up-to-date: rasterstats in /usr/local/lib/python3.6/dist-packages (0.15.0)
Requirement already up-to-date: wget in /usr/local/lib/python3.6/dist-packages (3.2)
Requirement already up-to-date: pygeos in /usr/local/lib/python3.6/dist-packages (0.8)


In [2]:
!apt install libspatialindex-dev
!pip install --upgrade rtree

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libspatialindex-dev is already the newest version (1.8.5-5).
0 upgraded, 0 newly installed, 0 to remove and 11 not upgraded.
Requirement already up-to-date: rtree in /usr/local/lib/python3.6/dist-packages (0.9.4)


In [1]:
import os
import pandas as pd
import geopandas as gpd
import numpy as np
import rasterio
import gdal
from shapely.geometry import shape
from rasterstats import zonal_stats
import fiona
fiona.supported_drivers['KML'] = 'rw'
import wget
import datetime

/usr/local/lib/python3.6/dist-packages/geopandas/_compat.py:88: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.8.1-CAPI-1.13.3). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


In [4]:
from google.colab import files
uploaded = files.upload()

In [2]:
## variables:
prefix='normal' # config prefix

dossier="./" ### working directory
min_slope = 70  ### valeur de pente minimale pour etre identifi� comme un cliff (en degr�s)
surface_min = 100 ### surface minimale pour �tre consid�r� comme une falaise int�ressante (en metres carr�s)
hauteur_min = 20 ### hauteur minimale pour �tre ocnsid�r�e comme une falaise int�ressante (en m�tres)


In [3]:
def process() :
    region=f"{dossier}/region_proj.shp"  ## path vers le shapes de regions
    index=f"{dossier}/Index_MNT20k.shp" ### path vers index des feuillets
    geol=f"{dossier}/geol_simple.shp" ### path vers le fichier des zones geologiques du Quebec
    carriere= f"{dossier}/carrieres.shp" ### path vers le fichier des carrieres du Quebec
    distance_carriere=1000 ### distance minimale d une carriere pour etre consid�r�e comme une falaise

    ### selection par region administrative ou shapefile custom
    regions_a_traiter=['15'] ### num�ro des r�gions administrative � traiter
    custom_shape=["F:\GL_Antoine\webmap\clip.shp"] #### path vers les shapefile custom
    type_process='region'   ### DOIT ETRE 'shape' ou 'region'
    if type_process=='region':

        for r in regions_a_traiter:

            dest_folder = f'{dossier}/{r}'  ### path pour fichier de destination (doit pouvoir contenir envrion 5Gb)
            print('processing region {}...'.format(r))

            os.makedirs(dest_folder, exist_ok=True)

            # liste de feuillet deja traites ou downloade,  si jamais le script ou  le FTP plante en cours de route
            liste=os.listdir(dest_folder)
            liste_fait=[]
            liste_download=[]
            for f in liste:
                if f [-4:] == '.shp':
                    f=f.split('_')[-1]
                    f=f[:-4]
                    liste_fait.append(f)
                elif f [-4:] == '.tif':
                    f=f.split('_')[-1]
                    f=f[:-4]
                    liste_download.append(f)
                else:
                    continue
            region=gpd.read_file(region)

            region=region[region['RES_CO_REG']==r]

            index=gpd.read_file(index)

            tiles=gpd.overlay(index, region, how='intersection')


    elif type_process=='shape':

        for shapefile in custom_shape:
            r=shapefile.split('\\')[-1].replace('.shp', '')
            dest_folder = fr'{dossier}\{r}'  ### path pour fichier de destination (doit pouvoir contenir envrion 5Gb)
            print('processing shapefile {}...'.format(r))
            os.makedirs(dest_folder, exist_ok=True)

            # liste de feuillet deja traites ou downloade,  si jamais le script ou  le FTP plante en cours de route
            liste=os.listdir(dest_folder)
            liste_fait=[]
            liste_download=[]
            for f in liste:
                if f [-4:] == '.shp':
                    f=f.split('_')[-1]
                    f=f[:-4]
                    liste_fait.append(f)
                elif f [-4:] == '.tif':
                    f=f.split('_')[-1]
                    f=f[:-4]
                    liste_download.append(f)
                else:
                    continue
            region=gpd.read_file(shapefile)
            region=region.to_crs({'init': 'epsg:4326'})
            index=gpd.read_file(index)
            tiles=gpd.overlay(index, region, how='intersection')

    else:
        print('!!"type_process" variable must be "shape" or "region"!!')

    start_timer = datetime.datetime.now()

    tiles=tiles['No_tuile2']

    print('total of {} tiles to process'.format(len(tiles)))
    count_tile=0

    ## pour chaque feuillet a linterieur de la region
    # reverse order
    for t in tiles.iloc[::-1]:
        count_tile = count_tile + 1
        print(f'\n\n---------------------  Tile {count_tile} / {len(tiles)} ---------------------')
        start_timer_tile = datetime.datetime.now()
        t_1=str(t[:-3])
        t_2=t[-3:]
        #print(t_2)
        if t_2=='202':
            t_2='NE'
        elif t_2=='201':
            t_2 = 'NO'
        elif t_2 == '102':
            t_2 = 'SE'
        elif t_2 == '101':
            t_2 = 'SO'
        else:
            print('weird orientation')
            pass
        feuillet=t_1+t_2
        print(f'-------------------------  {feuillet} -------------------------\n\n')

        ### download mnt if needed
        print('processing tile {}...'.format(feuillet))
        link ='ftp://transfert.mffp.gouv.qc.ca/Public/Diffusion/DonneeGratuite/Foret/IMAGERIE/Produits_derives_LiDAR/{}/{}/MNT_{}.tif'.format(feuillet[:3], feuillet, feuillet)  ### lien vers le MNT lidar du MFFP

        mnt = '{}/MNT_{}.tif'.format(dest_folder, feuillet)

        if feuillet in liste_fait:
            print(' {} deja fait'.format(feuillet))
            continue
        else:
            if feuillet not in liste_download:
                try:
                    if os.path.isfile(mnt):
                        print('file already downloaded:' + mnt)
                    else:
                        print('downloading file MNT_{}.tif....'.format(feuillet))
                        wget.download(link, out=mnt)
                except Exception as err:
                    print('mnt pas disponible: ' + err)
                    continue
            else:
                print('mnt already downloaded, processing...')
                pass

            ## calcul de pente et des orientations de pente
            slopes = 'slopes.tif'
            aspect = 'aspect.tif'
            gdal.DEMProcessing(slopes, mnt, "slope")
            gdal.DEMProcessing(aspect, mnt, "aspect")

            ## identidfication des portion avec une pente assez prononc�e pour �tre consdid�r�e comme une falaise
            raster = rasterio.open(slopes)
            band = raster.read(1)

            band = band.clip((min_slope-1), min_slope)

            band[band == (min_slope-1)] = 0
            band[band == min_slope] = 1

            # print(len(band[band==1]))

            if len(band[band==1]) == 0:
                print('no slope of more than {} degrees identified in tile {}, skipping'.format(feuillet, min_slope))
                raster.close()
                os.remove(slopes)
                os.remove(aspect)
                continue

            mask = band != 0

            ## retrait des petits bouts de falaises insignifiants

            mypoly = []
            for vec in rasterio.features.shapes(band, mask=mask, transform=raster.transform):
                mypoly.append(shape(vec[0]))

            gdf = gpd.GeoDataFrame(mypoly, crs=raster.crs)

            gdf = gdf.rename(columns={0: "geometry"}).set_geometry('geometry')

            gdf['surface'] = gdf['geometry'].area

            gdf = gdf[gdf['surface'] > surface_min]

            if len(gdf) == 0:
                print('no large cliffs identified in tile {}, skipping'.format(feuillet))
                raster.close()
                os.remove(slopes)
                os.remove(aspect)
                continue

            ## retrait des falaises pas assez hautes
            gdf['hauteur'] = np.nan
            count = 0

            for f in range(len(gdf)):
                count = count + 1
                count_low = count - 1
                feature = gdf.iloc[f, 0]
                minimum = zonal_stats(feature, mnt, stats=['min', 'max'], nodata='0', all_touched=True)
                height = minimum[0]['max'] - minimum[0]['min']
                gdf.iloc[f, 2] = height

            gdf = gdf[gdf['hauteur'] > hauteur_min]
            gdf['surface'] = gdf['surface'].round(3)
            gdf['hauteur'] = gdf['hauteur'].round(3)

            if len(gdf) == 0:
                print('no high cliffs identified in tile {}, skipping'.format(feuillet))
                raster.close()
                os.remove(slopes)
                os.remove(aspect)
                continue

            print('{} cliffs identified'.format(len(gdf)))

            ## identification de l'orientaiton des falaises
            gdf['orientation'] = np.nan
            count = 0

            for f in range(len(gdf)):
                count = count + 1
                count_low = count - 1
                feature = gdf.iloc[f, 0]
                aspect_cat = zonal_stats(feature, aspect, stats=['median'], nodata='0', all_touched=True)
                def cat(x):
                    if x >= 337.5 and x < 360:
                        return 'N'
                    if x >= 0 and x < 22.5:
                        return 'N'
                    if x >= 22.5 and x < 67.5:
                        return 'NE'
                    if x >= 67.5 and x < 112.5:
                        return 'E'
                    if x >= 112.5 and x < 157.5:
                        return 'SE'
                    if x >= 157.5 and x < 202.5:
                        return 'S'
                    if x >= 202.5 and x < 247.5:
                        return 'SO'
                    if x >= 247.5 and x < 292.5:
                        return 'O'
                    if x >= 292.5 and x < 337.5:
                        return 'NO'
                    else:
                        return 'NaN'
                value=round((aspect_cat[0]['median']), 3)
                value=cat(value)
                gdf.iloc[f, 3] = value

            ## calcul de la pente moyenne
            gdf['pente_moyenne'] = np.nan
            count = 0

            for f in range(len(gdf)):
                count = count + 1
                count_low = count - 1
                feature = gdf.iloc[f, 0]
                av_slope = zonal_stats(feature, slopes, stats=['mean'], nodata='0', all_touched=True)
                av_slope = av_slope[0]['mean']
                gdf.iloc[f, 4] = av_slope

            ## extraction du feuillet en shapefile et kml
            gdf.to_file(f'{prefix}_falaises_region_{feuillet}.shp', driver='ESRI Shapefile')
            gdf.to_file(f'{prefix}_falaises_region_{feuillet}.kml', driver='KML')

            raster.close()
            os.remove(slopes)
            os.remove(aspect)
            end_timer_tile = datetime.datetime.now()
            time_elapsed_tile = end_timer_tile - start_timer_tile

            print(f'tile {feuillet} processed in {time_elapsed_tile}')


    ## joindre tous les feuillet de la region en un seul gdf
    liste_shp=os.listdir(dest_folder)
    count=0
    gdf_all = 0
    for f in liste_shp:
        if f[-3:]=='shp':
            count=count+1
            path=os.path.join(dest_folder, f)
            gdf=gpd.read_file(path)
            gdf = gdf.to_crs({'init': 'epsg:4326'})
            if count == 1:
                gdf_all = gdf
            else:
                gdf_all = pd.concat([gdf_all, gdf], sort=False)
        else:
            continue

    if not gdf_all:
        return

    ### clip avec les limites de la région ou du shape
    gdf_all=gpd.overlay(gdf_all, region, how='intersection')

    ### enlever les carrieres
    print('removing quarries...')
    carriere_gdf=gpd.read_file(carriere)
    carriere_gdf=gdf.to_crs({'init': 'epsg:32618'})
    buff=gdf.buffer(distance_carriere)
    buff=gpd.GeoDataFrame(gpd.GeoSeries(buff), crs={'init': 'epsg:32618'})
    buff= buff.rename(columns={0:'geometry'}).set_geometry('geometry')
    buff=buff.to_crs({'init': 'epsg:4326'})
    gdf_all = gpd.overlay(gdf_all, buff, how="difference")
    geol_gdf=gpd.read_file(geol)
    geol_gdf=geol_gdf.to_crs({'init': 'epsg:4326'})

    ### joindre le type de roche
    print('joining geology info...')
    gdf_all = gpd.sjoin(gdf_all, geol_gdf, how="inner", op='intersects')

    ### enregistrer le tout en shapefile et KML
    print('saving all tiles in one shapefile...')
    gdf_all.to_file('falaises_region_{}.shp'.format(r), driver='ESRI Shapefile')
    gdf_all.to_file('falaises__region_{}.kml'.format(r), driver='KML')

    end_timer = datetime.datetime.now()
    time_elapsed = end_timer - start_timer
    print(f'Cliffs identified for region {r} in {time_elapsed}')

In [ ]:
process()

processing region 15...
total of 127 tiles to process


---------------------  Tile 1 / 127 ---------------------
-------------------------  31g08NE -------------------------


processing tile 31g08NE...
mnt already downloaded, processing...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:162: FutureWarning: Assigning CRS to a GeoDataFrame without a geometry column is now deprecated and will not be supported in the future.


no large cliffs identified in tile 31g08NE, skipping


---------------------  Tile 2 / 127 ---------------------
-------------------------  31h05NO -------------------------


processing tile 31h05NO...
mnt already downloaded, processing...
no high cliffs identified in tile 31h05NO, skipping


---------------------  Tile 3 / 127 ---------------------
-------------------------  31g09SE -------------------------


processing tile 31g09SE...
mnt already downloaded, processing...
no large cliffs identified in tile 31g09SE, skipping


---------------------  Tile 4 / 127 ---------------------
-------------------------  31g09SO -------------------------


processing tile 31g09SO...
mnt already downloaded, processing...
no high cliffs identified in tile 31g09SO, skipping


---------------------  Tile 5 / 127 ---------------------
-------------------------  31g10SE -------------------------


processing tile 31g10SE...
mnt already downloaded, processing...
no large cliffs identified in tile 31g

In [ ]:
prefix = 'strict'
dossier="./" ### working directory
min_slope = 85  ### valeur de pente minimale pour etre identifi� comme un cliff (en degr�s)
surface_min = 300 ### surface minimale pour �tre consid�r� comme une falaise int�ressante (en metres carr�s)
hauteur_min = 50 ### hauteur minimale pour �tre ocnsid�r�e comme une falaise int�ressante (en m�tres)

process()

In [ ]:
prefix = 'high_cliffs'
dossier="./" ### working directory
min_slope = 70  ### valeur de pente minimale pour etre identifi� comme un cliff (en degr�s)
surface_min = 200 ### surface minimale pour �tre consid�r� comme une falaise int�ressante (en metres carr�s)
hauteur_min = 70 ### hauteur minimale pour �tre ocnsid�r�e comme une falaise int�ressante (en m�tres)

process()

In [ ]:
prefix = '85_15'
dossier="./" ### working directory
min_slope = 85  ### valeur de pente minimale pour etre identifi� comme un cliff (en degr�s)
surface_min = 100 ### surface minimale pour �tre consid�r� comme une falaise int�ressante (en metres carr�s)
hauteur_min = 15 ### hauteur minimale pour �tre ocnsid�r�e comme une falaise int�ressante (en m�tres)

process()

In [ ]:
!zip -r /content/contents.zip /content
files.download("/content/contents{prefix}.zip")

In [ ]:
if True:
  !ls
else:
  !echo 'here'
  print('not working...')